In [33]:
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from wow_api.external.blizz_api.auth import BlizzardApiAuth

load_dotenv()

True

In [2]:
api_auth = BlizzardApiAuth()
token = api_auth.get_token()

made new token: access_token='USeyQIFtglJFINWwkEYQRyQaxfhrRScGnH' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 3, 29, 15, 23, 35, 895189, tzinfo=datetime.timezone.utc)
access_token='USeyQIFtglJFINWwkEYQRyQaxfhrRScGnH' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 3, 29, 15, 23, 35, 895189, tzinfo=datetime.timezone.utc)


In [3]:
class ConnectedRealm(BaseModel):
    href: str


class ConnectedRealmsIndexResponse(BaseModel):
    connected_realms: list[ConnectedRealm]


In [6]:
url = 'https://us.api.blizzard.com/data/wow/connected-realm/index?namespace=dynamic-us&locale=en_US'

connected_realms_res = requests.get(url, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

connected_realms = ConnectedRealmsIndexResponse.parse_obj(connected_realms_res.json())

with open('./data/realms.json', 'w') as realms_file:
    realms_file.write(connected_realms.json(indent=2))

In [12]:
class LocaleText(BaseModel):
    en_US: str
    es_MX: str
    pt_BR: str
    de_DE: str
    en_GB: str
    es_ES: str
    fr_FR: str
    it_IT: str
    ru_RU: str
    ko_KR: str
    zh_TW: str
    zh_CN: str


class Status(BaseModel):
    type: str
    name: LocaleText


class Population(BaseModel):
    type: str
    name: LocaleText


class Key(BaseModel):
    href: str


class Region(BaseModel):
    key: Key
    name: LocaleText
    id: int


class RealmConnectedRealm(BaseModel):
    href: str


class Type(BaseModel):
    type: str
    name: LocaleText


class Realm(BaseModel):
    id: int
    region: Region
    connected_realm: RealmConnectedRealm
    name: LocaleText
    category: LocaleText
    locale: str
    timezone: str
    type: Type
    is_tournament: bool
    slug: str


class MythicLeaderboards(BaseModel):
    href: str


class Auctions(BaseModel):
    href: str


class ConnectedRealmResponse(BaseModel):
    id: int
    has_queue: bool
    status: Status
    population: Population
    realms: list[Realm]
    mythic_leaderboards: MythicLeaderboards
    auctions: Auctions

In [13]:
realm_res = requests.get(connected_realms.connected_realms[0].href, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

connected_realm = ConnectedRealmResponse.parse_obj(realm_res.json())

In [23]:
class Key(BaseModel):
    href: str


class CurrentLeaderboard(BaseModel):
    key: Key
    name: LocaleText
    id: int


class MythicLeaderboardsResponse(BaseModel):
    current_leaderboards: list[CurrentLeaderboard]

In [24]:
mythic_leaderboards_res = requests.get(connected_realm.mythic_leaderboards.href, headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

leader_boards = MythicLeaderboardsResponse.parse_obj(mythic_leaderboards_res.json())

In [42]:
connected_realm.mythic_leaderboards.href

'https://us.api.blizzard.com/data/wow/connected-realm/121/mythic-leaderboard/?namespace=dynamic-us'

In [34]:






class Map(BaseModel):
    name: LocaleText
    id: int





class Realm(BaseModel):
    key: Key
    id: int
    slug: str


class Profile(BaseModel):
    name: str
    id: int
    realm: Realm


class Faction(BaseModel):
    type: str


class Specialization(BaseModel):
    key: Key
    id: int


class Member(BaseModel):
    profile: Profile
    faction: Faction
    specialization: Specialization


class Color(BaseModel):
    r: int
    g: int
    b: int
    a: float


class MythicRating(BaseModel):
    color: Color
    rating: float


class LeadingGroup(BaseModel):
    ranking: int
    duration: int
    completed_timestamp: int
    keystone_level: int
    members: list[Member]
    mythic_rating: MythicRating


class KeystoneAffix(BaseModel):
    key: Key
    name: LocaleText
    id: int


class CurrentAffix(BaseModel):
    keystone_affix: KeystoneAffix
    starting_level: int


class KeyLeaderboardResponse(BaseModel):
    map: Map
    period: int
    period_start_timestamp: int
    period_end_timestamp: int
    connected_realm: ConnectedRealm
    leading_groups: list[LeadingGroup]
    keystone_affixes: list[CurrentAffix]
    map_challenge_mode_id: int
    name: LocaleText

In [35]:
key_leaderboard_res = requests.get('https://us.api.blizzard.com/data/wow/connected-realm/121/mythic-leaderboard/2/period/899?namespace=dynamic-us', headers= {
        'Authorization': f'Bearer {token.access_token}'
    })

In [36]:
key_leaderboard = KeyLeaderboardResponse.parse_obj(key_leaderboard_res.json())

In [38]:
key_leaderboard.leading_groups[0].members[0].profile.name


'Gokgokgokgok'

In [51]:
res = requests.get('https://us.api.blizzard.com/data/wow/playable-race/29?namespace=static-10.0.7_48520-us',
                        headers= {
                            'Authorization': f'Bearer {token.access_token}'
                        })

In [52]:
res.status_code

200

In [53]:
with open('./data/race-res.json', 'w') as spec_file:
    spec_file.write(
        res.text
    )
